In [ ]:
# **Music Genre Classification using Neural Networks with Audio Signal Processing**


# **Step 1: Install Necessary Libraries**

# You'll need to install a variety of libraries, including:

!pip install numpy pandas librosa scikit-learn keras tensorflow

# **Step 2: Data Collection**

# We need a dataset of music clips from various genres. GTZAN dataset is widely used for this kind of task. It contains 100 clips per genre, each of 30 seconds duration, totaling 10 genres.

# **Step 3: Feature Extraction**

# We use Librosa library to extract features from the music files. The features we will use include:

# - Mel-frequency cepstral coefficients (MFCCs)
# - Tempo
# - Spectral contrast
# - Mel-scaled spectrogram

# Here's how to process audio files and extract features:


import librosa
import numpy as np

def extract_features(audio_path):
    y, sr = librosa.load(audio_path)
    mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40).T,axis=0)
    tempo, _ = librosa.beat.beat_track(y, sr=sr)
    contrast = np.mean(librosa.feature.spectral_contrast(y=y, sr=sr))
    mel = np.mean(librosa.feature.melspectrogram(y, sr=sr))

    return mfccs, tempo, contrast, mel


# **Step 4: Creating the Dataset**

# Combine features from all audio files to create a dataset, and assign corresponding genre labels to each data point:


import os
import pandas as pd

genres = 'blues classical country disco hiphop jazz metal pop reggae rock'
genres = genres.split()

data = []

for genre in genres:
    for filename in os.listdir(f'./genres/{genre}'):
        songname = f'./genres/{genre}/{filename}'
        mfccs, tempo, contrast, mel = extract_features(songname)
        data.append([mfccs, tempo, contrast, mel, genre])


# **Step 5: Data Preprocessing**

# The data must be pre-processed before feeding into the model. The labels are encoded into integers and the input features are scaled:

from sklearn.preprocessing import LabelEncoder, StandardScaler

# Converting data into pandas DataFrame
data = pd.DataFrame(data, columns=['mfccs', 'tempo', 'contrast', 'mel', 'genre'])

# Encoding labels
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)

# Scaling the Feature columns
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))


# **Step 6: Splitting the Data**

# We split the data into training and test sets:


from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


# **Step 7: Building the Model**

# We use Keras to build a Neural Network model with several layers:


from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))

model.add(layers.Dense(128, activation='relu'))

model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(10, activation='softmax'))


# **Step 8: Compiling the model**

# We compile the model, setting the optimizer, loss and metrics:


model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


# **Step 9: Training the Model**

# Now we train the model using our training data:


history = model.fit(X_train,
                    y_train,
                    epochs=20,
                    batch_size=128)


# **Step 10: Testing the model**

# Finally, we evaluate the model using our test data:


test_loss, test_acc = model.evaluate(X_test,y_test)


# **Step 11: Make Predictions**

# You ideally use a different set of music files to make predictions:


predictions = model.predict(X_test)

